In [14]:
import pandas as pd

df = pd.read_csv('/content/dataset_with_normal_sboxes.csv', skip_blank_lines=False)
split_index = df.index[df.isnull().all(axis=1)][0]


plaintext_df = df.iloc[:split_index]
ciphertext_df = df.iloc[split_index + 2:]


plaintext_data = plaintext_df.to_numpy()
ciphertext_data = ciphertext_df.to_numpy()


<ipython-input-14-fa096455466c>:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/dataset_with_normal_sboxes.csv', skip_blank_lines=False)


In [15]:
from sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp = train_test_split(ciphertext_data, plaintext_data, train_size=80000, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=10000, random_state=42)


X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [16]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np


X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


model = Sequential()
model.add(LSTM(64, input_shape=(1, 64), return_sequences=False))
model.add(Dense(64, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))


X_train shape: (80000, 1, 64)
y_train shape: (80000, 64)
X_val shape: (10000, 1, 64)
y_val shape: (10000, 64)
X_test shape: (10000, 1, 64)
y_test shape: (10000, 64)
Epoch 1/10
1250/1250 [==============================] - 9s 4ms/step - loss: 0.6756 - accuracy: 0.0022 - val_loss: 0.6686 - val_accuracy: 0.0015
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6644 - accuracy: 5.5000e-04 - val_loss: 0.6624 - val_accuracy: 0.0012
Epoch 3/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6592 - accuracy: 4.8750e-04 - val_loss: 0.6585 - val_accuracy: 2.0000e-04
Epoch 4/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6559 - accuracy: 3.8750e-04 - val_loss: 0.6560 - val_accuracy: 4.0000e-04
Epoch 5/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.6537 - accuracy: 3.7500e-04 - val_loss: 0.6545 - val_accuracy: 0.0000e+00
Epoch 6/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6523 - ac

In [ ]:

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Overall Test Accuracy: {test_accuracy}")
y_pred = model.predict(X_test)


y_pred_binary = (y_pred > 0.5).astype(int)
bit_accuracies = (y_pred_binary == y_test).mean(axis=0)


313/313 [==============================] - 1s 3ms/step - loss: 0.6938 - accuracy: 0.0295
Overall Test Accuracy: 0.029500000178813934
313/313 [==============================] - 1s 2ms/step


In [ ]:

bit_accuracy_df = pd.DataFrame({
    'Bit Position': [f'Bit {i}' for i in range(64)],
    'Accuracy': bit_accuracies
})


bit_accuracy_df.to_csv('bit_accuracies.csv', index=False)
print("Bit-wise accuracies have been saved to 'bit_accuracies.csv'.")


Bit-wise accuracies have been saved to 'bit_accuracies.csv'.


**The below code should be ran after we get both the csv files, and then used for comparison**

In [11]:
import pandas as pd

# Load the data
normal_sbox_df = pd.read_csv('/content/Normal S-Box results.csv')
changed_sbox_df = pd.read_csv('/content/Changed S-Box Results.csv')

# Combine the data for comparison
comparison_df = normal_sbox_df.copy()
comparison_df['Accuracy_Changed_Sbox'] = changed_sbox_df['Accuracy']

# Calculate mean accuracies
mean_accuracy_normal = comparison_df['Accuracy'].mean()
mean_accuracy_changed = comparison_df['Accuracy_Changed_Sbox'].mean()

# Calculate accuracy differences
comparison_df['Accuracy_Difference'] = comparison_df['Accuracy_Changed_Sbox'] - comparison_df['Accuracy']

# Summary statistics
improved_bits = (comparison_df['Accuracy_Difference'] > 0).sum()
degraded_bits = (comparison_df['Accuracy_Difference'] < 0).sum()
unchanged_bits = (comparison_df['Accuracy_Difference'] == 0).sum()

# Print the detailed comparison and conclusions
print(f"Mean accuracy with normal S-box: {mean_accuracy_normal:.4f}")
print(f"Mean accuracy with changed S-box: {mean_accuracy_changed:.4f}")
print(f"Bits with improved accuracy: {improved_bits}")
print(f"Bits with degraded accuracy: {degraded_bits}")
print(f"Bits with unchanged accuracy: {unchanged_bits}")
print("\nDetailed Comparison:\n", comparison_df)


# Determine which bits have the most significant changes
comparison_df['Significant_Change'] = comparison_df['Accuracy_Difference'].apply(lambda x: 'Improved' if x > 0 else 'Degraded' if x < 0 else 'Unchanged')
print("\nBits with Significant Changes:\n", comparison_df[['Bit Position', 'Significant_Change', 'Accuracy_Difference']])


if mean_accuracy_changed > mean_accuracy_normal:
    print("\nOverall, the LSTM model performed better with the changed S-box.")
else:
    print("\nOverall, the LSTM model performed worse with the changed S-box.")



Mean accuracy with normal S-box: 0.5383
Mean accuracy with changed S-box: 0.5006
Bits with improved accuracy: 32
Bits with degraded accuracy: 32
Bits with unchanged accuracy: 0

Detailed Comparison:
    Bit Position  Accuracy  Accuracy_Changed_Sbox  Accuracy_Difference
0         Bit 0    0.4965                 0.5082               0.0117
1         Bit 1    0.4970                 0.4995               0.0025
2         Bit 2    0.4911                 0.5074               0.0163
3         Bit 3    0.4952                 0.4974               0.0022
4         Bit 4    0.4999                 0.5013               0.0014
..          ...       ...                    ...                  ...
59       Bit 59    0.8490                 0.5005              -0.3485
60       Bit 60    0.9185                 0.5097              -0.4088
61       Bit 61    0.8516                 0.4971              -0.3545
62       Bit 62    0.8554                 0.4944              -0.3610
63       Bit 63    0.8535     